# Parsing Statistics of Income (SOI) Tax Tables from the IRS
John Mays | maysj@omb.nyc.gov | Created: 03/11/25 | Last Updated: 03/12/25

Data is from the "Individual income tax returns with exemptions and itemized deductions > Publication 1304" category on the [IRS.gov website](https://www.irs.gov/statistics/soi-tax-stats-individual-statistical-tables-by-size-of-adjusted-gross-income).

In [1]:
import pandas as pd
import re
from pathlib import Path
from tqdm import tqdm # cool arabic word here: taqadum (meaning: progress) = تقدم

In [2]:
data_directory = Path('../data')

## Helper Methods:

In [3]:
def find_total_returns_label_cells(sheet:pd.DataFrame) -> list:
    indices = []
    for column in sheet.columns:
        col_matches = sheet[column].str.match(
            r"^taxable[, ]*returns[, ]*total", flags=re.IGNORECASE, na=False
        )
        row_indices = list(sheet.index[col_matches])
        if row_indices:
            indices += [(r, column) for r in row_indices]
    return indices

#### Finding Likely Header Rows:
In the data, the header rows are displayed like: (1) (2) (3)... or perhaps (11) (12) (13)...

These can show up in one of two ways when imported into pandas: as negative or positive values.  But, either way, it is a repeating series of values horizontally decrementing or incrementing by 1, so it should be easy to find.


In [4]:
def find_likely_header_rows(sheet: pd.DataFrame, threshold=5) -> list:
    """
    This function scans sheet for rows that are likely full of numeric headers.
    In the data, the header rows are displayed like: (1) (2) (3)...
    These can show up in one of two ways when imported into pandas: as negative
    or positive values.  But, either way, it is a repeating series of values
    horizontally decrementing or incrementing by 1, so it should be easy to find.

    Arguments:
        sheet: the dataframe to be scanned through
        threshold: the number of 1 incrrements/decrements to id a row as a 
        likely header
    Returns:
        list of row indexes
    """
    likely_header_rows = []
    for row_index in sheet.index:
        row = sheet.iloc[row_index, :]
        one_increments = 0
        prev_value = None
        for value in row:
            if isinstance(value, int) and isinstance(prev_value, int):
                if abs(value - prev_value) == 1:
                    one_increments += 1
            prev_value = value
        if one_increments > threshold:
            likely_header_rows.append(row_index)
    return likely_header_rows

In [5]:
def find_tax_year(file_name:str) -> int:
    """
    This function takes the IRS format for filenames and returns
    the tax year that it is from in an integer format.
    """
    yy = file_name[0:2]
    cc = '19' if int(yy) >= 90 else '20'
    return int(cc + yy)

In [6]:
def assemble_header(segments:list) -> str:
    """
    This function takes the list of text found likely to be part of the header
    and concatenates in clean header format with underscores    
    """
    segments_text_only = [s for s in segments if isinstance(s, str)]
    header = '_'.join(segments_text_only)
    header = re.sub(r"\[|\]|-| |_|\\|\/|//|\n", "_", header) # replace special chars
    header = re.sub(r"\%", '_percent_', header) # replace %
    header = '_'.join([s for s in header.split('_') if s != '']) # de-duplicate underscores
    header = header.lower()
    return header

In [7]:
def supplement_inadequate_text_headers(num_to_text_headers: dict) -> dict:
    for num_header, text_header in numeric_to_text_headers.items():
        if text_header == 'amount':
            # extract prefix info from the col before it:
            try:
                preceeding_col_index = -(abs(num_header)-1) if num_header < 0 else abs(num_header)-1
                preceeding_col_name = num_to_text_headers[preceeding_col_index]
                search_result = re.search('number', preceeding_col_name)
                if search_result:
                    prefix_end_index = search_result.span()[0]
                    prefix = preceeding_col_name[0:prefix_end_index]
                    num_to_text_headers[num_header] = prefix + text_header
            except KeyError:
                pass
    return num_to_text_headers

In [8]:
def find_total_returns(sheet:pd.DataFrame, total_returns_label_cells:list) -> dict:
    indices_and_tr_values = {}
    for total_returns_cell_index in total_returns_label_cells:
        tr_row, tr_column = total_returns_cell_index
        total_returns_cell = sheet.iloc[*total_returns_cell_index]
        possible_numeric_value_indices = [(tr_row, col) for col in sheet.columns if col > tr_column]
        for index in possible_numeric_value_indices:
            value = sheet.iloc[*index]
            if isinstance(value, (int, float)): # then the value is numeric & valid
                indices_and_tr_values[index] = value
    return indices_and_tr_values

## Collecting all of the files into dataframes:

In [9]:
def collect_files(dir: Path) -> dict:
    sheets = {}
    for sheet_path in tqdm([path for path in data_directory.glob('*in21*.xl*')]):
        sheets[sheet_path.name] = pd.read_excel(sheet_path, header=None)
    return sheets

In [10]:
sheets = collect_files(data_directory)

100%|██████████| 30/30 [00:00<00:00, 47.98it/s]


## Parsing the sheets:

In [11]:
total_returns_dicts = {}

for sheet_name, sheet in sheets.items():
    tax_year = find_tax_year(sheet_name)
    print(f'Sheet: {sheet_name}, Year: {tax_year}')
    # finding cells with "total returns" -- the value we want
    total_returns_label_cells = find_total_returns_label_cells(sheet)
    # finding likely header rows:
    likely_header_rows = find_likely_header_rows(sheet, threshold=5)
    # finding the numeric values of the total returns:
    indices_and_tr_values = find_total_returns(sheet, total_returns_label_cells)
    # finding the total returns' corresponding headers:
    numeric_headers_and_values = {}
    for index, value in indices_and_tr_values.items():
        value_row, value_column = index
        # finds the closest header row that is above this value:
        corresponding_header_row = max([row for row in likely_header_rows if row < value_row])
        numeric_header = sheet.iloc[corresponding_header_row, value_column]
        numeric_headers_and_values[numeric_header] = value
    # finding indices for the numeric headers:
    indices_and_numeric_headers = {}
    for header_row_index in likely_header_rows:
        header_row = sheet.iloc[header_row_index, :]
        for potential_header_column, potential_header in zip(header_row.index, header_row):
            if isinstance(potential_header, int) and potential_header < 1000:
                header_index = (header_row_index, potential_header_column)
                indices_and_numeric_headers[header_index] = potential_header
    # associating the numeric to the text headers:
    numeric_to_text_headers = {}
    for header_index, numeric_header in indices_and_numeric_headers.items():
        header_row, header_col = header_index
        lesser_row_index = max(header_row-8, 0)
        greater_row_index = max(header_row, 0)
        text_header_segments = list(sheet.iloc[lesser_row_index:greater_row_index, header_col])
        numeric_to_text_headers[numeric_header] = assemble_header(text_header_segments)

    # patch some of the headers which could only find further title text one cell over:
    numeric_to_text_headers = supplement_inadequate_text_headers(numeric_to_text_headers)

    # Associating values with text headers:
    text_headers_and_values = {}
    all_numeric_headers = set(numeric_headers_and_values.keys()).union(numeric_to_text_headers.keys())
    for numeric_header in all_numeric_headers:
        try:
            text_header = numeric_to_text_headers[numeric_header]
            value = numeric_headers_and_values[numeric_header]
            text_headers_and_values[text_header] = value
        except KeyError:
            pass
    # add tax_year as a header/value pair as well:
    # text_headers_and_values['tax_year'] = tax_year
    total_returns_dicts[tax_year] = text_headers_and_values

Sheet: 00in21id.xls, Year: 2000
Sheet: 01in21id.xls, Year: 2001
Sheet: 02in21id.xls, Year: 2002
Sheet: 03in21id.xls, Year: 2003
Sheet: 04in21id.xls, Year: 2004
Sheet: 05in21id.xls, Year: 2005
Sheet: 06in21id.xls, Year: 2006
Sheet: 07in21id.xls, Year: 2007
Sheet: 08in21id.xls, Year: 2008
Sheet: 09in21id.xls, Year: 2009
Sheet: 10in21id.xls, Year: 2010
Sheet: 11in21id.xls, Year: 2011
Sheet: 12in21id.xls, Year: 2012
Sheet: 13in21id.xls, Year: 2013
Sheet: 14in21id.xls, Year: 2014
Sheet: 15in21id.xls, Year: 2015
Sheet: 16in21id.xls, Year: 2016
Sheet: 17in21id.xls, Year: 2017
Sheet: 18in21id.xls, Year: 2018
Sheet: 19in21id.xls, Year: 2019
Sheet: 20in21id.xls, Year: 2020
Sheet: 21in21id.xls, Year: 2021
Sheet: 22in21id.xls, Year: 2022
Sheet: 93in21id.xls, Year: 1993
Sheet: 94in21id.xls, Year: 1994
Sheet: 95in21id.xls, Year: 1995
Sheet: 96in21id.xls, Year: 1996
Sheet: 97in21.xls, Year: 1997
Sheet: 98in21id.xls, Year: 1998
Sheet: 99in21id.xls, Year: 1999


In [12]:
total_returns = pd.DataFrame.from_records(total_returns_dicts)

In [13]:
total_returns = total_returns.sort_index(ascending=True)

In [14]:
total_returns.head()

,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
1_amount,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
adjusted_gross_income_less_deficit,2.209986e+09,2.311111e+09,2.510127e+09,2.777795e+09,3.095049e+09,3.416485e+09,3.795660e+09,4.227597e+09,4.075101e+09,3.959239e+09,...,5.751944e+09,6.216995e+09,6.524826e+09,6.525290e+09,7.176740e+09,4.154248e+09,4.112979e+09,4.130732e+09,5.160191e+09,4.737747e+09
agi_limitation_amount,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,4.017652e+07,4.127643e+07,4.636444e+07,2.529774e+07,2.542303e+07,2.179364e+07,2.322260e+07,2.500844e+07
agi_limitation_number_of_returns,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,6.069784e+06,6.216417e+06,7.352854e+06,3.440448e+06,3.385403e+06,2.865346e+06,2.781750e+06,2.987128e+06
alimony_received_amount,2.603022e+06,2.693727e+06,2.678863e+06,2.807874e+06,2.987752e+06,3.293131e+06,3.380582e+06,4.233701e+06,4.513831e+06,4.468002e+06,...,5.716311e+06,6.502334e+06,6.406219e+06,6.591953e+06,6.792223e+06,5.008615e+06,5.355087e+06,4.394875e+06,3.663338e+06,3.401942e+06


In [15]:
total_returns.to_excel(data_directory.joinpath('total_returns.xlsx'))